In [1]:
import os
import json
import pickle
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
autocast_questions = json.load(open('/content/gdrive/My Drive/autocast_questions.json')) # from the Autocast dataset
test_questions = json.load(open('/content/gdrive/My Drive/autocast_competition_test_set.json'))
test_ids = [q['id'] for q in test_questions]

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00


In [6]:

# Remove examples with Ids in the test set from the training set
filtered_train_data = [example for example in autocast_questions if example["id"] not in test_ids and example["answer"] is not None]

# Save the filtered training set to a new file
with open("filtered_train_data.json", "w") as f:
    json.dump(filtered_train_data, f)


from transformers import AutoTokenizer, AutoModelWithLMHead, T5ForConditionalGeneration, Trainer, TrainingArguments
import json

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-qasc")
model = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-qasc")
# Load the autocast dataset


f = open('/content/gdrive/My Drive/filtered_train_data.json')
json_obj = json.load(f)


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [7]:
import pandas as pd
print(len(json_obj))
print(len(json_obj[0]))
train_dataset = pd.DataFrame(json_obj)
column_names = list(train_dataset.keys())
print(column_names)

2797
14
['question', 'id', 'background', 'publish_time', 'close_time', 'tags', 'source_links', 'prediction_count', 'forecaster_count', 'answer', 'choices', 'status', 'qtype', 'crowd']


In [8]:
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

def get_response(question, context, max_length=64):
  input_text = 'question: %s  context: %s' % (question, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return cleanhtml(tokenizer.decode(output[0]))

print(get_response("Where is the capital of China?", "China is in east Asian"))
t1 = train_dataset.loc[train_dataset['id'] == 'M402'].index[0]
a1 = train_dataset.loc[t1, 'answer']
q1 = train_dataset.loc[t1, 'question']
b1 = train_dataset.loc[t1, 'background']
print(a1)
print(q1)
print(b1)
print(get_response(q1, b1))

 Beijing
0.42049
How many Federally Insured Commercial Banking Institutions will there be at the end of 2017?
The U.S. has seen a continued consolidation of the Banking Industry over the past 70+ Years. According to the FDIC, there were over 14,000 FDIC Insured Commercial banks in 1934. Over the next 50 years, the Country maintained similar levels of Commercial Banks (i.e., over 14,000 FDIC Insured Commercial banks in 1984). However, over the past 30+ years, this number has shrunk rapidly through bank mergers and failures. As of 3rd quarter of 2016, year end, there are just 5,170 banks in the United States; the lowest year end total on record. Conversely, banks balance sheets have been growing rapidly over the same time horizon. According to the same FDIC report, domestic deposits at these banks as of Q3 2016 are at the highest levels ever, at over $10.5 trillion. The reasoning for this consolidation are wide ranging, including impacts from the Great Recession (and other past recession